<a href="https://colab.research.google.com/github/nunocesarsa/SENSECO_School_2021/blob/main/ColabNotebooks/SENSECO_05_Downloading_S2_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing google earth engine package

In [ ]:
#importing Earth Engine package
import ee

In [ ]:
#then we need to authenticate
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Connecting to Google drive

In [ ]:
#and now we link the colab also to the google drive so we can use it to store the outputs
from google.colab import drive
drive.mount('/content/drive')

#Loading packages

In [ ]:
import pandas as pd


#1) Loading a shapefile

 - here we use geopandas to load a shapefile first
 - then we create a custom function that converts an input file to a list of coordinates
 - then we looad this list using the ee.Geometry.Polygon



In [ ]:
#installing needed package
!pip install geopandas

In [ ]:
import geopandas as gpd

CLBJ_shp = gpd.read_file("/content/drive/MyDrive/SENSECO/Shapefiles/CLBJ Boundary_Square_Small.shp")

#auxiliar function to convert to json
def coords_from_gpd(shape_in):
  return eval(shape_in.to_json())['features'][0]['geometry']['coordinates']


#loading to GEE - polygon cant be very complex it seems 
CLBJ_gee =  ee.Geometry.Polygon(coords_from_gpd(CLBJ_shp))

#2) Search collection

In [ ]:
tilelist = ['14SPB','14SPC']
tilelist = ['14SPC']

#this command creates an object which is already filtered to the aoi
s2_collection = (ee.ImageCollection("COPERNICUS/S2_SR") #selects the L8 Surface reflectance product in GEE
                 .select(['B2','B3','B4',#'B8A'
                          'B5','B6','B7',
                          'B8A','B11','B12'
                          ]) #selects the bands of interest
                 .filter(ee.Filter.date('2016-01-01','2020-03-01')) #filters on the time - you can play around with this to select different dates..
                 .filterBounds(CLBJ_gee) #filters on the Aoi we have created above
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5))
                 .filter(ee.Filter.inList('MGRS_TILE', tilelist)))

#checking the number of images
count = s2_collection.size()
print('Nr of images in collection: ', str(count.getInfo())+'\n')

## Optional: exploring the metadata

In [ ]:
mid_step = s2_collection.toList(count)
s2_img = ee.Image(mid_step.get(0))
s2_img_meta = s2_img.getInfo()
s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B1')

250.068464074

In [ ]:
#exploring the file names
for i in range(count.getInfo()):
  mid_step = s2_collection.toList(count) #this is an internal step to help us loop on the data itself
  s2_img = ee.Image(mid_step.get(i)) #this fetches ith image on the collection 
  s2_img_meta = s2_img.getInfo()   #this creates an object which has only the metadata 
  print('At position',i,'is Product ID: ',s2_img_meta.get('properties',{}).get('PRODUCT_ID'))


## Optional: Visualizing one image

For this, we import folium pacakge, and create a specific function to assist. This is to make a prettier and more explorable map than the normal.

(this was copy pasted, probably there are other ways of doing the same).

This tutorial gives an excellent run down example of the many options in folium

https://spatial.utk.edu/maps/ee-api-folium-setup.html

In [ ]:
#compare 2 dates

#in here, we select an image of the list - in python lists start at 0
mid_step = s2_collection.toList(count)
s2_img_1 = ee.Image(mid_step.get(0))
s2_img_2 = ee.Image(mid_step.get(1))

#this command clips the image based on the original poly area
s2_img_clip = s2_img_1.clip(CLBJ_gee)

# Import the Folium library. This library is to assist in visualizations
import folium

# Import the Image function from the IPython.display module. 
from IPython.display import Image

# Define a method for displaying Earth Engine image tiles to folium map.
    #the next code snippet i stole from someone
    #this is a simplified version of the original function
def add_ee_layer(self, ee_image_object, vis_params, name):
  
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)



# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

#starting location of the map
my_map = folium.Map(location=[33.3,-97.6], zoom_start=12) #this had to be tried one by one... feels daf

# Set visualization parameters. Here is we can create different visualizations of our data
visParams = {"bands": ['B4', 'B3', 'B2'],"gain": '0.1, 0.1, 0.1',"scale":20}
visParams_nir = {"bands": ['B8A', 'B4', 'B3'],"gain": '0.1, 0.1, 0.1',"scale":20}

# Add the image to the folium map
my_map.add_ee_layer(s2_img_1, visParams, 'RGB Date 1')
my_map.add_ee_layer(s2_img_clip, visParams_nir, 'NIR composite - clipped')
my_map.add_ee_layer(s2_img_2, visParams, 'RGB Date 2')

#adding a layer control
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

#3) Downloading

In [ ]:

#df_inc_angles = pd.DataFrame(columns=["Imagename","Azimuth","Zenith"])
df_angles = pd.DataFrame(columns=["Imagename","Sen_Azimuth","Sen_Zenith","Sol_Azimuth","Sol_Zenith"])

#this is an internal step to help us loop on the data itself
mid_step = s2_collection.toList(count)

for i in range(count.getInfo()):
#for i in range(1):


  #print(i)

  s2_img = ee.Image(mid_step.get(i)) #this fetches ith image on the collection 
  s2_img_meta = s2_img.getInfo()   #this creates an object which has only the metadata 
  imagename = s2_img_meta.get('properties',{}).get('PRODUCT_ID')
  #print('At position',i,'is Product ID: ',imagename)

  #calculating a mean azimuth and mean zenith from all bands used
  mean_azi = (s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B2')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B3')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B4')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B5')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B6')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B7')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B11')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B12'))/9

  mean_zen = (s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_ZENITH_ANGLE_B2')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_ZENITH_ANGLE_B3')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_ZENITH_ANGLE_B4')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_ZENITH_ANGLE_B5')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_ZENITH_ANGLE_B6')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_ZENITH_ANGLE_B7')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_ZENITH_ANGLE_B8A')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_ZENITH_ANGLE_B11')+\
              s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_ZENITH_ANGLE_B12'))/9
  

  #storing the angle information into a table
  df_angles = df_angles.append({"Imagename":imagename,
                                "Sen_Azimuth":mean_azi,
                                "Sen_Zenith":mean_zen,
                                "Sol_Azimuth":s2_img_meta.get('properties',{}).get('MEAN_SOLAR_AZIMUTH_ANGLE'),
                                "Sol_Zenith" :s2_img_meta.get('properties',{}).get('MEAN_SOLAR_ZENITH_ANGLE')},ignore_index=True)

  #for checkup if needed
  #print(mean_azi)
  #print(s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B3'))
  #print(mean_zen)
  #print(s2_img_meta.get('properties',{}).get('MEAN_INCIDENCE_ZENITH_ANGLE_B2'))
  
  

  #these are task commands to be sent to GEE
  task_config = {
      'image': s2_img,
      'fileFormat': 'GeoTIFF',
      'folder': 'SENSECO_S2Data', #Meijendel folder 
      'fileNamePrefix': imagename[0:19] + "_T14SPC", #careful with the tile name, if its changed then its wrong
      'description': "clipped area",
      'scale':20,
      'region':CLBJ_gee
  }

  #print("Google Earth engine task started")
  task = ee.batch.Export.image.toDrive(**task_config )
  task.start()

  #creates a loop that is infinite until the its finished
  k=1
  flag = task.status()['state']
  while flag != 'COMPLETED':
    
    flag = task.status()['state']
    #print(k,flag)

    #this is to check the status every N iterations
    if k == 30:
      print(flag)
      k=1
    
    k=k+1

    if flag == 'FAILED':

      print("GEE data cropping failed - proceeding to next image")
      print("Failed: "+ out_s2_name)
      flag = 'COMPLETED' 
  print(flag)

  print("Saved at: " + "/content/drive/MyDrive/SENSECO/S2Data/" + imagename + ".tif")


  

In [ ]:
df_angles.to_csv("/content/drive/MyDrive/SENSECO_S2Data/AngleInformation.csv",sep=",",decimal=".")
df_angles.to_csv("/content/drive/MyDrive/SENSECO_S2Data/AngleInformation_csvNuno.csv",sep=";",decimal=".")